In [1]:
import os
import numpy as np
import rasterio
from geopy.distance import geodesic
from tqdm import tqdm
# Paths to input and output folders
input_folder = "H:\\Global_tree_cover\\2000extent"



# Get list of tif files
tif_files = [f for f in os.listdir(input_folder) if f.endswith(".tif")]

"""
# Loop through each tif file in the input folder
for i, filename in enumerate(tif_files, desc="Processing files"):
    tqdm.write(f"Processing file {i+1}/{len(tif_files)}: {filename}")
"""

'\n# Loop through each tif file in the input folder\nfor i, filename in enumerate(tif_files, desc="Processing files"):\n    tqdm.write(f"Processing file {i+1}/{len(tif_files)}: {filename}")\n'

In [2]:
import re

def extract_first_digit(string):
    """Extract the first sequence of digits from the string."""
    pattern = r'\d+'  # Match one or more numbers
    match = re.search(pattern, string)
    
    if match:
        return int(match.group())  # Return the matched number (as an integer)
    else:
        return None  # If no number is matched, return None

def extract_second_digit(string):
    """Extract the second sequence of digits from the string."""
    pattern = r'\d+'  # Match one or more numbers
    matches = re.findall(pattern, string)
    
    if len(matches) >= 2:
        return int(matches[1])  # Return the second number matched (as an integer)
    else:
        return None  # If no number is matched, return None

def extract_first_letter(string):
    """Extract the first English letter in the string."""
    count = 0  # Used to count the number of English letters encountered

    for char in string:
        if char.isalpha():  # Check if the character is a letter
            count += 1

            if count == 1:
                return char

def extract_second_letter(string):
    """Extract the second English letter in the string."""
    count = 0  # Used to count the number of English letters encountered

    for char in string:
        if char.isalpha():  # Check if the character is a letter
            count += 1

            if count == 2:
                return char

    return None  # If the second letter is not found, return None

# Define the file list
#tif_files = ['00N_000E.tif']
file = tif_files[15]
print(tif_files[15])

# Extract digits and letters from the file name
first_digit = extract_first_digit(file)
second_digit = extract_second_digit(file)
first_letter = extract_first_letter(file)
second_letter = extract_second_letter(file)

if first_letter == 'N':
    up_first_digit = int(first_digit) + 10
    up_first_letter = 'N'
    if first_digit == 0:
        up_first_digit = 10
        up_first_letter = 'N'
        down_first_digit = 10
        down_first_letter = 'S'
        up_second_digit = second_digit
        down_second_digit = second_digit
        up_second_letter = second_letter
        down_second_letter = second_letter
    else:
        up_first_digit = first_digit + 10
        down_first_digit = int(first_digit) - 10
        down_first_letter = 'N'
        up_first_letter = 'N'
        up_second_digit = second_digit
        down_second_digit = second_digit
        up_second_letter = second_letter
        down_second_letter = second_letter
elif first_letter == 'S':
    if first_digit == 10:
        up_first_digit = 0
        up_first_letter = 'N'
        down_first_digit = 20
        down_first_letter = 'S'
        up_second_digit = second_digit
        down_second_digit = second_digit
        up_second_letter = second_letter
        down_second_letter = second_letter
    else:
        up_first_digit = int(first_digit) - 10
        up_first_letter = 'S'
        down_first_digit = int(first_digit) + 10
        down_first_letter = 'S'
        up_second_digit = second_digit
        down_second_digit = second_digit
        up_second_letter = second_letter
        down_second_letter = second_letter

# East or West
if second_letter == 'E':
    right_second_digit = second_digit + 10
    right_second_letter = second_letter
    left_second_digit = second_digit - 10
    left_second_letter = second_letter
    if second_digit == 0:
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = 10
        left_second_letter = 'W'
    if second_digit == 170:
        right_second_digit = 180
        right_second_letter = 'W'
        left_second_digit = second_digit - 10
        left_second_letter = 'E'
elif second_letter == 'W':
    right_second_digit = second_digit - 10
    right_second_letter = second_letter
    left_second_digit = second_digit + 10
    left_second_letter = second_letter
    if second_digit == 10:
        right_second_digit = 0
        right_second_letter = 'E'
        left_second_digit = 20
        left_second_letter = 'W'
    if second_digit == 180:
        right_second_digit = 170
        right_second_letter = 'W'
        left_second_digit = 170
        left_second_letter = 'E'
        
# Construct the file names for the surrounding files
north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

# Print out the file names
print(north_file_name)
print(south_file_name)
print(east_file_name)
print(west_file_name)

30S_130E.tif
20S_130E.tif
40S_130E.tif
30S_140E.tif
30S_120E.tif


In [9]:
# function of the neighbouring finding.

import re

def get_surrounding_files(file_name):
    """Given a file name, return the file names of the files corresponding to 
    the geographical locations to the north, south, east, and west."""

    def extract_first_digit(string):
        """Extract the first sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        match = re.search(pattern, string)
        return int(match.group()) if match else None

    def extract_second_digit(string):
        """Extract the second sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        matches = re.findall(pattern, string)
        return int(matches[1]) if len(matches) >= 2 else None

    def extract_first_letter(string):
        """Extract the first English letter in the string."""
        for char in string:
            if char.isalpha(): return char

    def extract_second_letter(string):
        """Extract the second English letter in the string."""
        count = 0
        for char in string:
            if char.isalpha(): 
                count += 1
                if count == 2: return char

    first_digit = extract_first_digit(file_name)
    second_digit = extract_second_digit(file_name)
    first_letter = extract_first_letter(file_name)
    second_letter = extract_second_letter(file_name)

    if first_letter == 'N':
        up_first_digit = int(first_digit) + 10
        up_first_letter = 'N'
        if first_digit == 0:
            up_first_digit = 10
            up_first_letter = 'N'
            down_first_digit = 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = first_digit + 10
            down_first_digit = int(first_digit) - 10
            down_first_letter = 'N'
            up_first_letter = 'N'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
    elif first_letter == 'S':
        if first_digit == 10:
            up_first_digit = 0
            up_first_letter = 'N'
            down_first_digit = 20
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = int(first_digit) - 10
            up_first_letter = 'S'
            down_first_digit = int(first_digit) + 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter

    # East or West
    if second_letter == 'E':
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = second_digit - 10
        left_second_letter = second_letter
        if second_digit == 0:
            right_second_digit = second_digit + 10
            right_second_letter = second_letter
            left_second_digit = 10
            left_second_letter = 'W'
        if second_digit == 170:
            right_second_digit = 180
            right_second_letter = 'W'
            left_second_digit = second_digit - 10
            left_second_letter = 'E'
    elif second_letter == 'W':
        right_second_digit = second_digit - 10
        right_second_letter = second_letter
        left_second_digit = second_digit + 10
        left_second_letter = second_letter
        if second_digit == 10:
            right_second_digit = 0
            right_second_letter = 'E'
            left_second_digit = 20
            left_second_letter = 'W'
        if second_digit == 180:
            right_second_digit = 170
            right_second_letter = 'W'
            left_second_digit = 170
            left_second_letter = 'E'
        
    north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
    south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
    east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
    west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

    return north_file_name, south_file_name, east_file_name, west_file_name

north, south, east, west = get_surrounding_files('30N_130W.tif')
print(north)
print(south)
print(east)
print(west)

40N_130W.tif
20N_130W.tif
30N_120W.tif
30N_140W.tif


In [10]:
import os
import rasterio

def get_border_pixels(file_name):
    with rasterio.open(file_name) as dataset:
        band1 = dataset.read(1)  # assuming you want to read the first band

        north_pixels = band1[0, :]  # North border pixels
        south_pixels = band1[-1, :]  # South border pixels
        east_pixels = band1[:, -1]  # East border pixels
        west_pixels = band1[:, 0]  # West border pixels

    return north_pixels, south_pixels, east_pixels, west_pixels

def process_files(input_folder, central_file):
    # Calculate file names of the surrounding files
    north_file, south_file, east_file, west_file = get_surrounding_files(central_file)

    # List of files and corresponding border pixels
    file_list = [(north_file, 1), (south_file, 0), (east_file, 3), (west_file, 2)]

    # Dictionary to store file existence flags and border pixels
    border_data = {}

    # Get the border pixels of the central file
    central_file_path = os.path.join(input_folder, central_file)
    if os.path.exists(central_file_path):
        central_pixels = get_border_pixels(central_file_path)
    else:
        print(f"Warning: {central_file} does not exist. Skipping this file.")
        return None

    for file, border_index in file_list:
        file_path = os.path.join(input_folder, file)
        # Check if the file exists
        if os.path.exists(file_path):
            # For north and south files, we need to consider the south and north borders of the central file, respectively.
            if border_index in [0, 1]:
                central_border_pixels = central_pixels[1 - border_index]  
            # For east and west files, we need to consider the west and east borders of the central file, respectively.
            else:  
                central_border_pixels = central_pixels[5 - border_index]
            border_data[file] = {
                "exists": True, 
                "pixels": get_border_pixels(file_path)[border_index], 
                "central_pixels": central_border_pixels
            }
            print(f"Complete: {file} exists. Extract the broder.")
        else:
            border_data[file] = {"exists": False}
            print(f"Warning: {file} does not exist. Skipping this file.")

    return border_data


# Usage:
input_folder = "H:\\Global_tree_cover\\2000extent"
central_file = '40N_020E.tif'
border_data = process_files(input_folder, central_file)
north_file, south_file, east_file, west_file = get_surrounding_files(central_file)
# Now border_data is a dictionary that holds both the existence flag and the border pixels (if the file exists) for each file.

Complete: 50N_020E.tif exists. Extract the broder.
Complete: 30N_020E.tif exists. Extract the broder.
Complete: 40N_030E.tif exists. Extract the broder.
Complete: 40N_010E.tif exists. Extract the broder.


In [11]:
central_file_path = os.path.join(input_folder, central_file)
central_file_path

'H:\\Global_tree_cover\\2000extent\\40N_020E.tif'

In [12]:
print(border_data[north_file]['exists'])

sum(border_data[north_file]['pixels'])

True


12535

In [13]:
print(border_data[south_file]['pixels'].shape)

sum(border_data[south_file]['pixels'])

(40000,)


0

In [14]:
print(border_data[east_file]['pixels'].shape)
sum(border_data[east_file]['pixels'])

(40000,)


2429

In [15]:
print(border_data[west_file]['pixels'].shape)
sum(border_data[west_file]['pixels'])

(40000,)


171

In [16]:
import rasterio
import numpy as np
central_file_path_edge = os.path.join(input_folder, central_file)
dataset = rasterio.open(central_file_path)

width = dataset.width
height = dataset.height

upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
left_pixels = dataset.read(1, window=((0, height), (0, 1)))
right_pixels = dataset.read(1, window=((0, height), (width-1, width)))


left_pixels = np.reshape(left_pixels, (height,))
right_pixels = np.reshape(right_pixels, (height,))
upper_pixels = np.reshape(upper_pixels, (height,))
lower_pixels = np.reshape(lower_pixels, (height,))

print("Upper pixels:", upper_pixels, sum(upper_pixels))
print("Lower pixels:", lower_pixels, sum(lower_pixels))
print("Left pixels:", left_pixels, sum(left_pixels))
print("Right pixels:", right_pixels, sum(right_pixels))

Upper pixels: [0 0 1 ... 0 0 0] 12489
Lower pixels: [0 0 0 ... 0 0 0] 0
Left pixels: [0 0 0 ... 0 0 0] 167
Right pixels: [0 0 0 ... 0 0 0] 2511


In [17]:
import rasterio
import numpy as np

input_folder_edge = "H:\\Global_tree_cover\\TIFoutput\\2000Edge"
central_file_path_edge = os.path.join(input_folder_edge, central_file)
dataset_edge = rasterio.open(central_file_path_edge)

upper_pixels_edge = dataset_edge.read(1, window=((0, 1), (0, width)))
lower_pixels_edge = dataset_edge.read(1, window=((height-1, height), (0, width)))
left_pixels_edge = dataset_edge.read(1, window=((0, height), (0, 1)))
right_pixels_edge = dataset_edge.read(1, window=((0, height), (width-1, width)))


left_pixels_edge = np.reshape(left_pixels_edge, (height,))
right_pixels_edge = np.reshape(right_pixels_edge, (height,))
upper_pixels_edge = np.reshape(upper_pixels_edge, (height,))
lower_pixels_edge = np.reshape(lower_pixels_edge, (height,))

print("Upper pixels:", upper_pixels_edge, sum(upper_pixels_edge))
print("Lower pixels:", lower_pixels_edge, sum(lower_pixels_edge))
print("Left pixels:", left_pixels_edge, sum(left_pixels_edge))
print("Right pixels:", right_pixels_edge, sum(right_pixels_edge))

Upper pixels: [ 0.       0.      77.00821 ...  0.       0.       0.     ] 68822.38343811035
Lower pixels: [0. 0. 0. ... 0. 0. 0.] 0.0
Left pixels: [0. 0. 0. ... 0. 0. 0.] 1413.0437660217285
Right pixels: [0. 0. 0. ... 0. 0. 0.] 16955.33346748352


In [18]:
import copy
left_pixels_edge_new = copy.copy(left_pixels_edge)
right_pixels_edge_new = copy.copy(right_pixels_edge)
upper_pixels_edge_new = copy.copy(upper_pixels_edge)
lower_pixels_edge_new = copy.copy(lower_pixels_edge)

In [19]:
from geopy.distance import geodesic

# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters

# Calculate the width of the top and bottom row of pixels
width_lookup = compute_width_lookup(dataset.height, dataset.transform)
top_row_width = width_lookup[0]
bottom_row_width = width_lookup[-1]

# Calculate the height of the left and right columns of pixels
left_column_height = compute_height(abs(dataset.transform[4]))
right_column_height = compute_height(abs(dataset.transform[0]))

# Print the width of the top and bottom row of pixels, and the height of the left and right columns of pixels
print("Top row width:", top_row_width)
print("Bottom row width:", bottom_row_width)
print("Left column height:", left_column_height)
print("Right column height:", right_column_height)

Top row width: 21.348464239647665
Bottom row width: 24.121509601861483
Left column height: 27.829872698318393
Right column height: 27.829872698318393


In [20]:
left_column_height

27.829872698318393

In [21]:
if border_data[north_file]['exists']:
    for i in range(len(border_data[north_file]['pixels'])):
        b = upper_pixels[i]
        if b == 0:
            continue
        a = (border_data[north_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            upper_pixels_edge_new[i] = upper_pixels_edge[i] + top_row_width
            print(upper_pixels_edge[i],upper_pixels_edge_new[i])

21.348465 42.69693
27.829872 49.178337
55.659744 77.00821
55.659744 77.00821
55.659744 77.00821
49.178337 70.5268
21.348465 42.69693
21.348465 42.69693
49.178337 70.5268
49.178337 70.5268
21.348465 42.69693
49.178337 70.5268
27.829872 49.178337
77.00821 98.356674
55.659744 77.00821
27.829872 49.178337
49.178337 70.5268
49.178337 70.5268
0.0 21.348465
0.0 21.348465
55.659744 77.00821
77.00821 98.356674
27.829872 49.178337
0.0 21.348465
27.829872 49.178337
0.0 21.348465
0.0 21.348465
27.829872 49.178337
0.0 21.348465
27.829872 49.178337
27.829872 49.178337
0.0 21.348465
27.829872 49.178337
0.0 21.348465
0.0 21.348465
27.829872 49.178337
77.00821 98.356674
27.829872 49.178337
27.829872 49.178337
27.829872 49.178337
27.829872 49.178337
0.0 21.348465
27.829872 49.178337
0.0 21.348465
0.0 21.348465
0.0 21.348465
0.0 21.348465
0.0 21.348465
0.0 21.348465
49.178337 70.5268
27.829872 49.178337
27.829872 49.178337
27.829872 49.178337
27.829872 49.178337
0.0 21.348465
27.829872 49.178337
49.17833

In [22]:
print("Upper pixels:", upper_pixels_edge, sum(upper_pixels_edge))
print("Upper pixels new:", upper_pixels_edge_new, sum(upper_pixels_edge_new))


Upper pixels: [ 0.       0.      77.00821 ...  0.       0.       0.     ] 68822.38343811035
Upper pixels new: [ 0.       0.      77.00821 ...  0.       0.       0.     ] 89594.43984985352


In [23]:
if border_data[south_file]['exists']:
    for i in range(len(border_data[south_file]['pixels'])):
        b = lower_pixels[i]
        if b == 0:
            continue
        a = (border_data[south_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            lower_pixels_edge_new[i] = lower_pixels_edge[i] + bottom_row_width
            print(lower_pixels_edge[i],lower_pixels_edge_new[i])

In [24]:
if border_data[west_file]['exists']:
    for i in range(len(border_data[west_file]['pixels'])):
        b = left_pixels[i]
        if b == 0:
            continue
        a = (border_data[west_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            left_pixels_edge_new[i] = left_pixels_edge[i] + left_column_height
            print(left_pixels_edge[i],left_pixels_edge_new[i])

70.53155 98.36142
49.241653 77.071526
0.0 27.829872
21.426579 49.25645
21.434244 49.26412
49.35009 77.17996
0.0 27.829872
49.350395 77.18027
70.87238 98.702255
0.0 27.829872
21.52806 49.357933
21.530062 49.359936
21.53429 49.364162


In [25]:
if border_data[east_file]['exists']:
    for i in range(len(border_data[east_file]['pixels'])):
        b = right_pixels[i]
        if b == 0:
            continue
        a = (border_data[east_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            right_pixels_edge_new[i] = right_pixels_edge[i] + right_column_height
            print(right_pixels_edge[i],right_pixels_edge_new[i])

21.354069 49.18394
21.355703 49.185577
21.359982 49.189854
0.0 27.829872
21.364494 49.194366
42.736374 70.566246
21.374989 49.20486
21.376078 49.20595
0.0 27.829872
0.0 27.829872
70.589874 98.41975
49.213642 77.04352
21.383923 49.213795
70.59857 98.428444
21.3913 49.221172
0.0 27.829872
0.0 27.829872
0.0 27.829872
21.404646 49.23452
21.404957 49.23483
21.407127 49.237
0.0 27.829872
0.0 27.829872
21.408602 49.238476
0.0 27.829872
0.0 27.829872
0.0 27.829872
0.0 27.829872
0.0 27.829872
0.0 27.829872
0.0 27.829872
49.252422 77.0823
42.852306 70.68218
21.427044 49.256916
0.0 27.829872
21.427277 49.25715
42.86578 70.695656
70.69813 98.528
70.69844 98.52831
21.436953 49.266827
21.437262 49.267136
21.437494 49.26737
70.70649 98.53636
21.454973 49.284847
21.456827 49.2867
21.462854 49.29273
21.46548 49.295353
0.0 27.829872
21.478914 49.30879
21.481075 49.310947
27.829872 55.659744
21.481537 49.31141
0.0 27.829872
49.313725 77.1436
70.799194 98.62907
42.97133 70.8012
21.487555 49.31743
42.97533

In [26]:
print("Upper pixels:", sum(upper_pixels_edge))
print("Upper pixels new:", sum(upper_pixels_edge_new))
print("lower pixels:", sum(lower_pixels_edge))
print("lower pixels new:", sum(lower_pixels_edge_new))
print("Right pixels:", sum(right_pixels_edge))
print("Right pixels new:", sum(right_pixels_edge_new))
print("Left pixels:", sum(left_pixels_edge))
print("Left pixels new:", sum(left_pixels_edge_new))

Upper pixels: 68822.38343811035
Upper pixels new: 89594.43984985352
lower pixels: 0.0
lower pixels new: 0.0
Right pixels: 16955.33346748352
Right pixels new: 24580.718635559082
Left pixels: 1413.0437660217285
Left pixels new: 1774.8321075439453


In [7]:
import os
import rasterio
import re
import numpy as np
import copy
from geopy.distance import geodesic

def get_surrounding_files(file_name):
    """Given a file name, return the file names of the files corresponding to 
    the geographical locations to the north, south, east, and west."""

    def extract_first_digit(string):
        """Extract the first sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        match = re.search(pattern, string)
        return int(match.group()) if match else None

    def extract_second_digit(string):
        """Extract the second sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        matches = re.findall(pattern, string)
        return int(matches[1]) if len(matches) >= 2 else None

    def extract_first_letter(string):
        """Extract the first English letter in the string."""
        for char in string:
            if char.isalpha(): return char

    def extract_second_letter(string):
        """Extract the second English letter in the string."""
        count = 0
        for char in string:
            if char.isalpha(): 
                count += 1
                if count == 2: return char

    first_digit = extract_first_digit(file_name)
    second_digit = extract_second_digit(file_name)
    first_letter = extract_first_letter(file_name)
    second_letter = extract_second_letter(file_name)

    if first_letter == 'N':
        up_first_digit = int(first_digit) + 10
        up_first_letter = 'N'
        if first_digit == 0:
            up_first_digit = 10
            up_first_letter = 'N'
            down_first_digit = 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = first_digit + 10
            down_first_digit = int(first_digit) - 10
            down_first_letter = 'N'
            up_first_letter = 'N'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
    elif first_letter == 'S':
        if first_digit == 10:
            up_first_digit = 0
            up_first_letter = 'N'
            down_first_digit = 20
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = int(first_digit) - 10
            up_first_letter = 'S'
            down_first_digit = int(first_digit) + 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter

    # East or West
    if second_letter == 'E':
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = second_digit - 10
        left_second_letter = second_letter
        if second_digit == 0:
            right_second_digit = second_digit + 10
            right_second_letter = second_letter
            left_second_digit = 10
            left_second_letter = 'W'
        if second_digit == 170:
            right_second_digit = 180
            right_second_letter = 'W'
            left_second_digit = second_digit - 10
            left_second_letter = 'E'
    elif second_letter == 'W':
        right_second_digit = second_digit - 10
        right_second_letter = second_letter
        left_second_digit = second_digit + 10
        left_second_letter = second_letter
        if second_digit == 10:
            right_second_digit = 0
            right_second_letter = 'E'
            left_second_digit = 20
            left_second_letter = 'W'
        if second_digit == 180:
            right_second_digit = 170
            right_second_letter = 'W'
            left_second_digit = 170
            left_second_letter = 'E'
        
    north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
    south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
    east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
    west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

    return north_file_name, south_file_name, east_file_name, west_file_name

################################################################################################################################


def get_border_pixels(file_name):
    with rasterio.open(file_name) as dataset:
        band1 = dataset.read(1)  # assuming you want to read the first band

        north_pixels = band1[0, :]  # North border pixels
        south_pixels = band1[-1, :]  # South border pixels
        east_pixels = band1[:, -1]  # East border pixels
        west_pixels = band1[:, 0]  # West border pixels

    return north_pixels, south_pixels, east_pixels, west_pixels

def process_files(input_folder, central_file):
    # Calculate file names of the surrounding files
    north_file, south_file, east_file, west_file = get_surrounding_files(central_file)

    # List of files and corresponding border pixels
    file_list = [(north_file, 1), (south_file, 0), (east_file, 3), (west_file, 2)]

    # Dictionary to store file existence flags and border pixels
    border_data = {}

    # Get the border pixels of the central file
    central_file_path = os.path.join(input_folder, central_file)
    if os.path.exists(central_file_path):
        central_pixels = get_border_pixels(central_file_path)
    else:
        print(f"Warning: {central_file} does not exist. Skipping this file.")
        return None

    for file, border_index in file_list:
        file_path = os.path.join(input_folder, file)
        # Check if the file exists
        if os.path.exists(file_path):
            # For north and south files, we need to consider the south and north borders of the central file, respectively.
            if border_index in [0, 1]:
                central_border_pixels = central_pixels[1 - border_index]  
            # For east and west files, we need to consider the west and east borders of the central file, respectively.
            else:  
                central_border_pixels = central_pixels[5 - border_index]
            border_data[file] = {
                "exists": True, 
                "pixels": get_border_pixels(file_path)[border_index], 
                "central_pixels": central_border_pixels
            }
        else:
            border_data[file] = {"exists": False}
    return border_data

################################################################################################################################

def read_border_pixels(file_path):
    dataset = rasterio.open(file_path)

    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################

def read_border_pixels_edge(file_path):
    dataset = rasterio.open(file_path)
    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################
# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters


# Usage:
input_folder = "H:\\Global_tree_cover\\2000extent"
central_file = '40N_020E.tif'
border_data = process_files(input_folder, central_file)
north_file, south_file, east_file, west_file = get_surrounding_files(central_file)
central_file_path = os.path.join(input_folder, central_file)
left_pixels, right_pixels, upper_pixels, lower_pixels = read_border_pixels(central_file_path)

input_folder_edge = "H:\\Global_tree_cover\\TIFoutput\\2000Edge"
central_file_path_edge = os.path.join(input_folder_edge, central_file)
left_pixels_edge, right_pixels_edge, upper_pixels_edge, lower_pixels_edge = read_border_pixels_edge(central_file_path_edge)
left_pixels_edge_new = copy.copy(left_pixels_edge)
right_pixels_edge_new = copy.copy(right_pixels_edge)
upper_pixels_edge_new = copy.copy(upper_pixels_edge)
lower_pixels_edge_new = copy.copy(lower_pixels_edge)

# Calculate the width of the top and bottom row of pixels
central_file_path_edge = os.path.join(input_folder, central_file)
dataset = rasterio.open(central_file_path)
width = dataset.width
height = dataset.height
width_lookup = compute_width_lookup(dataset.height, dataset.transform)
top_row_width = width_lookup[0]
bottom_row_width = width_lookup[-1]

# Calculate the height of the left and right columns of pixels
left_column_height = compute_height(abs(dataset.transform[4]))
right_column_height = compute_height(abs(dataset.transform[0]))


if border_data[north_file]['exists']:
    for i in range(len(border_data[north_file]['pixels'])):
        b = upper_pixels[i]
        if b == 0:
            continue
        a = (border_data[north_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            upper_pixels_edge_new[i] = upper_pixels_edge[i] + top_row_width
if border_data[south_file]['exists']:
    for i in range(len(border_data[south_file]['pixels'])):
        b = lower_pixels[i]
        if b == 0:
            continue
        a = (border_data[south_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            lower_pixels_edge_new[i] = lower_pixels_edge[i] + bottom_row_width
if border_data[west_file]['exists']:
    for i in range(len(border_data[west_file]['pixels'])):
        b = left_pixels[i] 
        if b == 0:
            continue
        a = (border_data[west_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            left_pixels_edge_new[i] = left_pixels_edge[i] + left_column_height
if border_data[east_file]['exists']:
    for i in range(len(border_data[east_file]['pixels'])):
        b = right_pixels[i]
        if b == 0:
            continue
        a = (border_data[east_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            right_pixels_edge_new[i] = right_pixels_edge[i] + right_column_height

print("Upper pixels:", sum(upper_pixels_edge))
print("Upper pixels new:", sum(upper_pixels_edge_new))
print("lower pixels:", sum(lower_pixels_edge))
print("lower pixels new:", sum(lower_pixels_edge_new))
print("Right pixels:", sum(right_pixels_edge))
print("Right pixels new:", sum(right_pixels_edge_new))
print("Left pixels:", sum(left_pixels_edge))
print("Left pixels new:", sum(left_pixels_edge_new))

Complete: 50N_020E.tif exists. Extract the broder.
Complete: 30N_020E.tif exists. Extract the broder.
Complete: 40N_030E.tif exists. Extract the broder.
Complete: 40N_010E.tif exists. Extract the broder.
Upper pixels: 68822.38343811035
Upper pixels new: 89594.43984985352
lower pixels: 0.0
lower pixels new: 0.0
Right pixels: 16955.33346748352
Right pixels new: 24580.718635559082
Left pixels: 1413.0437660217285
Left pixels new: 1774.8321075439453


In [27]:
# 创建新的TIFF文件来保存修改过的数据
new_tiff_path = "H:/Global_tree_cover/TIFoutput/new_tiff_file.tif"

with rasterio.open(central_file_path_edge, 'r') as src:
    # 读取现有数据
    data = src.read(1)

    # 在必要的位置替换像素值
    data[0, :] = upper_pixels_edge_new
    data[-1, :] = lower_pixels_edge_new
    data[:, 0] = left_pixels_edge_new
    data[:, -1] = right_pixels_edge_new

    # 创建新的TIFF文件
    with rasterio.open(new_tiff_path, 'w', driver='GTiff', 
                       height=src.height, width=src.width, count=1, 
                       dtype=str(data.dtype), 
                       crs=src.crs, transform=src.transform) as dst:
        dst.write(data, 1)


In [28]:
import os
import rasterio
import re
import numpy as np
import copy
from geopy.distance import geodesic

def get_surrounding_files(file_name):
    """Given a file name, return the file names of the files corresponding to 
    the geographical locations to the north, south, east, and west."""

    def extract_first_digit(string):
        """Extract the first sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        match = re.search(pattern, string)
        return int(match.group()) if match else None

    def extract_second_digit(string):
        """Extract the second sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        matches = re.findall(pattern, string)
        return int(matches[1]) if len(matches) >= 2 else None

    def extract_first_letter(string):
        """Extract the first English letter in the string."""
        for char in string:
            if char.isalpha(): return char

    def extract_second_letter(string):
        """Extract the second English letter in the string."""
        count = 0
        for char in string:
            if char.isalpha(): 
                count += 1
                if count == 2: return char

    first_digit = extract_first_digit(file_name)
    second_digit = extract_second_digit(file_name)
    first_letter = extract_first_letter(file_name)
    second_letter = extract_second_letter(file_name)

    if first_letter == 'N':
        up_first_digit = int(first_digit) + 10
        up_first_letter = 'N'
        if first_digit == 0:
            up_first_digit = 10
            up_first_letter = 'N'
            down_first_digit = 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = first_digit + 10
            down_first_digit = int(first_digit) - 10
            down_first_letter = 'N'
            up_first_letter = 'N'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
    elif first_letter == 'S':
        if first_digit == 10:
            up_first_digit = 0
            up_first_letter = 'N'
            down_first_digit = 20
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = int(first_digit) - 10
            up_first_letter = 'S'
            down_first_digit = int(first_digit) + 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter

    # East or West
    if second_letter == 'E':
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = second_digit - 10
        left_second_letter = second_letter
        if second_digit == 0:
            right_second_digit = second_digit + 10
            right_second_letter = second_letter
            left_second_digit = 10
            left_second_letter = 'W'
        if second_digit == 170:
            right_second_digit = 180
            right_second_letter = 'W'
            left_second_digit = second_digit - 10
            left_second_letter = 'E'
    elif second_letter == 'W':
        right_second_digit = second_digit - 10
        right_second_letter = second_letter
        left_second_digit = second_digit + 10
        left_second_letter = second_letter
        if second_digit == 10:
            right_second_digit = 0
            right_second_letter = 'E'
            left_second_digit = 20
            left_second_letter = 'W'
        if second_digit == 180:
            right_second_digit = 170
            right_second_letter = 'W'
            left_second_digit = 170
            left_second_letter = 'E'
        
    north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
    south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
    east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
    west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

    return north_file_name, south_file_name, east_file_name, west_file_name

################################################################################################################################


def get_border_pixels(file_name):
    with rasterio.open(file_name) as dataset:
        band1 = dataset.read(1)  # assuming you want to read the first band

        north_pixels = band1[0, :]  # North border pixels
        south_pixels = band1[-1, :]  # South border pixels
        east_pixels = band1[:, -1]  # East border pixels
        west_pixels = band1[:, 0]  # West border pixels

    return north_pixels, south_pixels, east_pixels, west_pixels

def process_files(input_folder, central_file):
    # Calculate file names of the surrounding files
    north_file, south_file, east_file, west_file = get_surrounding_files(central_file)

    # List of files and corresponding border pixels
    file_list = [(north_file, 1), (south_file, 0), (east_file, 3), (west_file, 2)]

    # Dictionary to store file existence flags and border pixels
    border_data = {}

    # Get the border pixels of the central file
    central_file_path = os.path.join(input_folder, central_file)
    if os.path.exists(central_file_path):
        central_pixels = get_border_pixels(central_file_path)
    else:
        print(f"Warning: {central_file} does not exist. Skipping this file.")
        return None

    for file, border_index in file_list:
        file_path = os.path.join(input_folder, file)
        # Check if the file exists
        if os.path.exists(file_path):
            # For north and south files, we need to consider the south and north borders of the central file, respectively.
            if border_index in [0, 1]:
                central_border_pixels = central_pixels[1 - border_index]  
            # For east and west files, we need to consider the west and east borders of the central file, respectively.
            else:  
                central_border_pixels = central_pixels[5 - border_index]
            border_data[file] = {
                "exists": True, 
                "pixels": get_border_pixels(file_path)[border_index], 
                "central_pixels": central_border_pixels
            }
        else:
            border_data[file] = {"exists": False}
    return border_data

################################################################################################################################

def read_border_pixels(file_path):
    dataset = rasterio.open(file_path)

    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################

def read_border_pixels_edge(file_path):
    dataset = rasterio.open(file_path)
    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################
# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters


# Usage:
input_folder = "H:\\Global_tree_cover\\2000extent"
central_file = '40N_020E.tif'
border_data = process_files(input_folder, central_file)
north_file, south_file, east_file, west_file = get_surrounding_files(central_file)
central_file_path = os.path.join(input_folder, central_file)
left_pixels, right_pixels, upper_pixels, lower_pixels = read_border_pixels(central_file_path)

input_folder_edge = "H:\\Global_tree_cover\\TIFoutput\\2000Edge"
central_file_path_edge = os.path.join(input_folder_edge, central_file)
left_pixels_edge, right_pixels_edge, upper_pixels_edge, lower_pixels_edge = read_border_pixels_edge(central_file_path_edge)
left_pixels_edge_new = copy.copy(left_pixels_edge)
right_pixels_edge_new = copy.copy(right_pixels_edge)
upper_pixels_edge_new = copy.copy(upper_pixels_edge)
lower_pixels_edge_new = copy.copy(lower_pixels_edge)

# Calculate the width of the top and bottom row of pixels
central_file_path_edge = os.path.join(input_folder, central_file)
dataset = rasterio.open(central_file_path)
width = dataset.width
height = dataset.height
width_lookup = compute_width_lookup(dataset.height, dataset.transform)
top_row_width = width_lookup[0]
bottom_row_width = width_lookup[-1]

# Calculate the height of the left and right columns of pixels
left_column_height = compute_height(abs(dataset.transform[4]))
right_column_height = compute_height(abs(dataset.transform[0]))


if border_data[north_file]['exists']:
    for i in range(len(border_data[north_file]['pixels'])):
        b = upper_pixels[i]
        if b == 0:
            continue
        a = (border_data[north_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            upper_pixels_edge_new[i] = upper_pixels_edge[i] + top_row_width
if border_data[south_file]['exists']:
    for i in range(len(border_data[south_file]['pixels'])):
        b = lower_pixels[i]
        if b == 0:
            continue
        a = (border_data[south_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            lower_pixels_edge_new[i] = lower_pixels_edge[i] + bottom_row_width
if border_data[west_file]['exists']:
    for i in range(len(border_data[west_file]['pixels'])):
        b = left_pixels[i] 
        if b == 0:
            continue
        a = (border_data[west_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            left_pixels_edge_new[i] = left_pixels_edge[i] + left_column_height
if border_data[east_file]['exists']:
    for i in range(len(border_data[east_file]['pixels'])):
        b = right_pixels[i]
        if b == 0:
            continue
        a = (border_data[east_file]['pixels'][i])
        if a*b == 1:
            continue
        else:
            right_pixels_edge_new[i] = right_pixels_edge[i] + right_column_height

with rasterio.open(central_file_path_edge, 'r+') as src:
    # Read the data
    data = src.read(1)

    # Replace the broder
    data[0, :] = upper_pixels_edge_new
    data[-1, :] = lower_pixels_edge_new
    data[:, 0] = left_pixels_edge_new
    data[:, -1] = right_pixels_edge_new
    # Write the data
    src.write(data, 1)

In [ ]:
import os
import rasterio
import re
import numpy as np
import copy
from geopy.distance import geodesic

################################################################################################################################
def get_surrounding_files(file_name):
    """Given a file name, return the file names of the files corresponding to 
    the geographical locations to the north, south, east, and west."""

    def extract_first_digit(string):
        """Extract the first sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        match = re.search(pattern, string)
        return int(match.group()) if match else None

    def extract_second_digit(string):
        """Extract the second sequence of digits from the string."""
        pattern = r'\d+'  # Match one or more numbers
        matches = re.findall(pattern, string)
        return int(matches[1]) if len(matches) >= 2 else None

    def extract_first_letter(string):
        """Extract the first English letter in the string."""
        for char in string:
            if char.isalpha(): return char

    def extract_second_letter(string):
        """Extract the second English letter in the string."""
        count = 0
        for char in string:
            if char.isalpha(): 
                count += 1
                if count == 2: return char

    first_digit = extract_first_digit(file_name)
    second_digit = extract_second_digit(file_name)
    first_letter = extract_first_letter(file_name)
    second_letter = extract_second_letter(file_name)

    if first_letter == 'N':
        up_first_digit = int(first_digit) + 10
        up_first_letter = 'N'
        if first_digit == 0:
            up_first_digit = 10
            up_first_letter = 'N'
            down_first_digit = 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = first_digit + 10
            down_first_digit = int(first_digit) - 10
            down_first_letter = 'N'
            up_first_letter = 'N'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
    elif first_letter == 'S':
        if first_digit == 10:
            up_first_digit = 0
            up_first_letter = 'N'
            down_first_digit = 20
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter
        else:
            up_first_digit = int(first_digit) - 10
            up_first_letter = 'S'
            down_first_digit = int(first_digit) + 10
            down_first_letter = 'S'
            up_second_digit = second_digit
            down_second_digit = second_digit
            up_second_letter = second_letter
            down_second_letter = second_letter

    # East or West
    if second_letter == 'E':
        right_second_digit = second_digit + 10
        right_second_letter = second_letter
        left_second_digit = second_digit - 10
        left_second_letter = second_letter
        if second_digit == 0:
            right_second_digit = second_digit + 10
            right_second_letter = second_letter
            left_second_digit = 10
            left_second_letter = 'W'
        if second_digit == 170:
            right_second_digit = 180
            right_second_letter = 'W'
            left_second_digit = second_digit - 10
            left_second_letter = 'E'
    elif second_letter == 'W':
        right_second_digit = second_digit - 10
        right_second_letter = second_letter
        left_second_digit = second_digit + 10
        left_second_letter = second_letter
        if second_digit == 10:
            right_second_digit = 0
            right_second_letter = 'E'
            left_second_digit = 20
            left_second_letter = 'W'
        if second_digit == 180:
            right_second_digit = 170
            right_second_letter = 'W'
            left_second_digit = 170
            left_second_letter = 'E'
        
    north_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(up_first_digit), up_first_letter, abs(second_digit), second_letter)
    south_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(down_first_digit), down_first_letter, abs(second_digit), second_letter)
    east_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(right_second_digit), right_second_letter)
    west_file_name = "{:02d}{}_{:03d}{}.tif".format(abs(first_digit), first_letter, abs(left_second_digit), left_second_letter)

    return north_file_name, south_file_name, east_file_name, west_file_name

################################################################################################################################


def get_border_pixels(file_name):
    with rasterio.open(file_name) as dataset:
        band1 = dataset.read(1)  # assuming you want to read the first band

        north_pixels = band1[0, :]  # North border pixels
        south_pixels = band1[-1, :]  # South border pixels
        east_pixels = band1[:, -1]  # East border pixels
        west_pixels = band1[:, 0]  # West border pixels

    return north_pixels, south_pixels, east_pixels, west_pixels

def process_files(input_folder, central_file):
    # Calculate file names of the surrounding files
    north_file, south_file, east_file, west_file = get_surrounding_files(central_file)

    # List of files and corresponding border pixels
    file_list = [(north_file, 1), (south_file, 0), (east_file, 3), (west_file, 2)]

    # Dictionary to store file existence flags and border pixels
    border_data = {}

    # Get the border pixels of the central file
    central_file_path = os.path.join(input_folder, central_file)
    if os.path.exists(central_file_path):
        central_pixels = get_border_pixels(central_file_path)
    else:
        print(f"Warning: {central_file} does not exist. Skipping this file.")
        return None

    for file, border_index in file_list:
        file_path = os.path.join(input_folder, file)
        # Check if the file exists
        if os.path.exists(file_path):
            # For north and south files, we need to consider the south and north borders of the central file, respectively.
            if border_index in [0, 1]:
                central_border_pixels = central_pixels[1 - border_index]  
            # For east and west files, we need to consider the west and east borders of the central file, respectively.
            else:  
                central_border_pixels = central_pixels[5 - border_index]
            border_data[file] = {
                "exists": True, 
                "pixels": get_border_pixels(file_path)[border_index], 
                "central_pixels": central_border_pixels
            }
        else:
            border_data[file] = {"exists": False}
    return border_data

################################################################################################################################

def read_border_pixels(file_path):
    dataset = rasterio.open(file_path)

    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################

def read_border_pixels_edge(file_path):
    dataset = rasterio.open(file_path)
    width = dataset.width
    height = dataset.height

    upper_pixels = dataset.read(1, window=((0, 1), (0, width)))
    lower_pixels = dataset.read(1, window=((height-1, height), (0, width)))
    left_pixels = dataset.read(1, window=((0, height), (0, 1)))
    right_pixels = dataset.read(1, window=((0, height), (width-1, width)))

    left_pixels = np.reshape(left_pixels, (height,))
    right_pixels = np.reshape(right_pixels, (height,))
    upper_pixels = np.reshape(upper_pixels, (width,))
    lower_pixels = np.reshape(lower_pixels, (width,))

    return left_pixels, right_pixels, upper_pixels, lower_pixels

################################################################################################################################
# Function to calculate lookup table for pixel widths
def compute_width_lookup(num_rows, transform):
    latitudes = [transform[5] + i * transform[4] for i in range(num_rows)]
    return [compute_width(lat, abs(transform[0])) for lat in latitudes]

# Function to calculate pixel width
def compute_width(lat, cellsize):
    point1 = (lat, 0)
    point2 = (lat, cellsize)
    return geodesic(point1, point2).meters

# Function to calculate pixel height
def compute_height(cellsize):
    point1 = (0, 0)
    point2 = (0, cellsize)
    return geodesic(point1, point2).meters
################################################################################################################################


# Paths for your folders
input_folder = "/mnt/cephfs/scratch/groups/chen_group/hangkai/2020extent"
output_folder_edge = "/mnt/cephfs/scratch/groups/chen_group/hangkai/2020Edge"
input_folder_edge = output_folder_edge
# Get all files in each folder
central_files = os.listdir(input_folder)
central_files_edge = os.listdir(output_folder_edge)

# Ensuring that the files in both folders correspond to each other
assert set(central_files) == set(central_files_edge)

# Loop through each file
for central_file in central_files:
    # Ensure the file is a .tif file
    if central_file.endswith('.tif'):
        print(f"Processing file: {central_file}")

################################################################################################################################
################################################################################################################################

        # Your processing code
        # replace 'process_files', 'read_border_pixels' and 'read_border_pixels_edge' with your actual functions
        # Usage:

        border_data = process_files(input_folder, central_file)
        north_file, south_file, east_file, west_file = get_surrounding_files(central_file)
        central_file_path = os.path.join(input_folder, central_file)
        left_pixels, right_pixels, upper_pixels, lower_pixels = read_border_pixels(central_file_path)

        central_file_path_edge = os.path.join(input_folder_edge, central_file)
        left_pixels_edge, right_pixels_edge, upper_pixels_edge, lower_pixels_edge = read_border_pixels_edge(central_file_path_edge)
        left_pixels_edge_new = copy.copy(left_pixels_edge)
        right_pixels_edge_new = copy.copy(right_pixels_edge)
        upper_pixels_edge_new = copy.copy(upper_pixels_edge)
        lower_pixels_edge_new = copy.copy(lower_pixels_edge)

        # Calculate the width of the top and bottom row of pixels
        central_file_path_edge = os.path.join(input_folder, central_file)
        dataset = rasterio.open(central_file_path)
        width = dataset.width
        height = dataset.height
        width_lookup = compute_width_lookup(dataset.height, dataset.transform)
        top_row_width = width_lookup[0]
        bottom_row_width = width_lookup[-1]

        # Calculate the height of the left and right columns of pixels
        left_column_height = compute_height(abs(dataset.transform[4]))
        right_column_height = compute_height(abs(dataset.transform[0]))


        if border_data[north_file]['exists']:
            for i in range(len(border_data[north_file]['pixels'])):
                b = upper_pixels[i]
                if b == 0:
                    continue
                a = (border_data[north_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    upper_pixels_edge_new[i] = upper_pixels_edge[i] + top_row_width
        if border_data[south_file]['exists']:
            for i in range(len(border_data[south_file]['pixels'])):
                b = lower_pixels[i]
                if b == 0:
                    continue
                a = (border_data[south_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    lower_pixels_edge_new[i] = lower_pixels_edge[i] + bottom_row_width
        if border_data[west_file]['exists']:
            for i in range(len(border_data[west_file]['pixels'])):
                b = left_pixels[i] 
                if b == 0:
                    continue
                a = (border_data[west_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    left_pixels_edge_new[i] = left_pixels_edge[i] + left_column_height
        if border_data[east_file]['exists']:
            for i in range(len(border_data[east_file]['pixels'])):
                b = right_pixels[i]
                if b == 0:
                    continue
                a = (border_data[east_file]['pixels'][i])
                if a*b == 1:
                    continue
                else:
                    right_pixels_edge_new[i] = right_pixels_edge[i] + right_column_height

        with rasterio.open(central_file_path_edge, 'r+') as src:
            # Read the data
            data = src.read(1)

            # Replace the border
            data[0, :] = upper_pixels_edge_new
            data[-1, :] = lower_pixels_edge_new
            data[:, 0] = left_pixels_edge_new
            data[:, -1] = right_pixels_edge_new

            # Multiply the data by the scale factor
            data = data * 10

            # Convert the data to np.uint16
            data = data.astype(np.uint16)

            # Write the data
            src.write(data, 1)